In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from config import gkey

In [2]:
#Loading csv file
csv_file = "cities.csv"
#DataFrame
weather_data_df = pd.read_csv(csv_file)

In [4]:
weather_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [11]:
#configure gmaps
gmaps.configure(gkey)

locations = weather_data_df[['Lat', 'Lng']]
weights = weather_data_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
vacation_df = weather_data_df

In [14]:
#Temperature adjustment
vacation_df = vacation_df.loc[(vacation_df["Max Temp"] < 80)]
vacation_df = vacation_df.loc[vacation_df["Max Temp"] > 70]

In [17]:
#Wind Speed Adjustment
vacation_df = vacation_df.loc[vacation_df["Wind Speed"] < 10]

#Cloudiness Adjustment
vacation_df = vacation_df.loc[vacation_df["Cloudiness"] == 0]

#Drop unwanted rows
vacation_df = vacation_df.dropna()

vacation_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [58]:
hotel_df = vacation_df

#dictionary to specify wants
params = {
    "radius": 5000,
    "types": "hotels",
    "key": gkey
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 70: ponta do sol.
Closest hotel is Capitólio.
------------
Retrieving Results for Index 88: cidreira.
Closest hotel is CIDREIRA.
------------
Retrieving Results for Index 138: morondava.
Closest hotel is Morondava.
------------
Retrieving Results for Index 139: vaini.
Closest hotel is Jagalbet.
------------
Retrieving Results for Index 176: nador.
Closest hotel is Nador.
------------
Retrieving Results for Index 297: mogok.
Closest hotel is Mogok.
------------
Retrieving Results for Index 444: birjand.
Closest hotel is Birjand.
------------


In [60]:
hotel_df
       
      


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Capitólio
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,CIDREIRA
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Morondava
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Jagalbet
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Nador
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Mogok
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,Birjand


In [61]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [67]:
# Create hotel symbol layer

markers = gmaps.marker_layer(locations)

hotel_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))